In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

In [2]:
test0 = ['##########',
         '#..O..O.O#',
         '#......O.#',
         '#.OO..O.O#',
         '#..O@..O.#',
         '#O#..O...#',
         '#O..O..O.#',
         '#.OO.O.OO#',
         '#....O...#',
         '##########',
         '',
         '<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^',
         'vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v',
         '><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<',
         '<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^',
         '^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><',
         '^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^',
         '>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^',
         '<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>',
         '^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>',
         'v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^']

test1 = ['########',
         '#..O.O.#',
         '##@.O..#',
         '#...O..#',
         '#.#.O..#',
         '#...O..#',
         '#......#',
         '########',
         '',
         '<^^>>>vv<v>>v<<']

test2 = ['#######',
         '#...#.#',
         '#.....#',
         '#..OO@#',
         '#..O..#',
         '#.....#',
         '#######',
         '',
         '<vv<<^^<<^^']

In [3]:
def parse_data(data, part=1):
    graph = []
    sequence = ''
    seq = False
    for line in data:
        line = line.strip()
        
        if len(line) == 0:
            seq = True
        elif seq:
            sequence += line
        else:
            if part == 1:
                graph.append(list(line))
            if part == 2:
                graph_line = []
                for char in list(line):
                    if char == '#':
                        graph_line.append('#')
                        graph_line.append('#')
                    elif char == '.':
                        graph_line.append('.')
                        graph_line.append('.')
                    elif char == 'O':
                        graph_line.append('[')
                        graph_line.append(']')
                    elif char =='@':
                        graph_line.append('@')
                        graph_line.append('.')
                graph.append(graph_line)
            
    graph = np.array(graph)
    start = np.where(graph == '@')
    graph[start] = '.'
    start = [start[0][0], start[1][0]]
            
    return np.array(graph), np.array(start), deque(list(sequence))

def print_graph(graph, pos):
    graph[pos[0],pos[1]] = '@'
    for line in graph:
        print(''.join(line))
    graph[pos[0],pos[1]] = '.'
    
def move_logic_part1(graph, pos, move, dirs, nxt_pos):
    if graph[nxt_pos[0],nxt_pos[1]] == '.':
            pos = nxt_pos
            
    elif graph[nxt_pos[0],nxt_pos[1]] == 'O':
        nxt_nxt_pos = deepcopy(nxt_pos)

        while True:
            nxt_nxt_pos += dirs[move]

            if graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '#':
                break

            elif graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '.':
                #move everything
                while nxt_nxt_pos[0] != nxt_pos[0] or nxt_nxt_pos[1] != nxt_pos[1]:
                    graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] = 'O'
                    nxt_nxt_pos -= dirs[move]

                graph[nxt_pos[0], nxt_pos[1]] = '.'
                pos = nxt_pos
                break
    return graph, pos

def move_logic_part2(graph, pos, move, dirs, nxt_pos):
    if graph[nxt_pos[0],nxt_pos[1]] == '.':
        return graph, nxt_pos
            
    elif graph[nxt_pos[0],nxt_pos[1]] == '#':
        return graph, pos
            
    elif move == '<' or move == '>':
        nxt_nxt_pos = deepcopy(nxt_pos)
    
        while True:
            nxt_nxt_pos += 2*dirs[move]
    
            if graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '#':
                break
    
            elif graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '.':
                #move everything
                while nxt_nxt_pos[0] != nxt_pos[0] or nxt_nxt_pos[1] != nxt_pos[1]:
                    graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] = graph[nxt_nxt_pos[0]-dirs[move][0],
                                                                 nxt_nxt_pos[1]-dirs[move][1]]
                    nxt_nxt_pos -= dirs[move]
    
                graph[nxt_pos[0], nxt_pos[1]] = '.'
                pos = nxt_pos
                break
                
    elif move == '^' or move == 'v':
        x = nxt_pos[0]
        dx = dirs[move][0]
        ys = {}
        
        nxt_ys = []
        nxt_ys.append(nxt_pos[1])
        if graph[nxt_pos[0],nxt_pos[1]] == '[':
            nxt_ys.append(nxt_pos[1]+1)
        else:
            nxt_ys.append(nxt_pos[1]-1)
        nxt_ys.sort()
        ys[x] = nxt_ys
            
        while True:
            nxt_x = x + dx
            
            if '#' in graph[nxt_x][ys[x]]:
                break
                
            nxt_y = []
            for y in ys[x]:
                if graph[nxt_x][y] == '[':
                    if y not in nxt_y:
                        nxt_y.append(y)
                    if y+1 not in nxt_y:
                        nxt_y.append(y+1)
                        
                elif graph[nxt_x][y] == ']':
                    if y not in nxt_y:
                        nxt_y.append(y)
                    if y-1 not in nxt_y:
                        nxt_y.append(y-1)
                        
            if len(nxt_y) > 0:
                nxt_y.sort()
                ys[nxt_x] = nxt_y
                x = nxt_x
            else:
                #move everything
                xes = list(ys.keys())
                if move == '^':
                    xes.sort()
                else: #move == 'v'
                    xes.sort(reverse=True)
                    
                for x in xes:
                    for y in ys[x]:
                        graph[x+dx][y] = graph[x][y]
                        if x-dx not in xes or y not in ys[x-dx]:
                            graph[x][y] = '.'
                pos = nxt_pos
                break
            
    return graph, pos

def run_robot(graph, pos, sequence, part=1, prnt=False):
    dirs = {'^':np.array([-1,0]), 'v':np.array([1,0]), '>':np.array([0,1]), '<':np.array([0,-1])}
    
    if prnt:
        print('Initial state')
        print_graph(graph, pos)
        print()
    
    while len(sequence):
        move = sequence.popleft()
        nxt_pos = pos + dirs[move]
        
        if part == 1:
            graph, pos = move_logic_part1(graph, pos, move, dirs, nxt_pos)
        elif part == 2:
            graph, pos = move_logic_part2(graph, pos, move, dirs, nxt_pos)
        
        if prnt:
            print('Move:', move)
            print_graph(graph, pos)
            print()
                    
    return graph

def gps_score(graph):
    boxes = np.where(np.logical_or(graph == 'O', graph == '['))
    score = 0
    
    for i in range(0, len(boxes[0])):
        score += (boxes[0][i]*100) + boxes[1][i]
        
    return score
        
def run(data, prnt=False, part=1):
    if part == 1 or part == 3:
        graph, start, sequence = parse_data(data)
        graph = run_robot(graph, start, sequence, prnt=prnt)
        score = gps_score(graph)
        print('Part 1 result:', score)
    
    if part == 2 or part == 3:
        graph, start, sequence = parse_data(data, part=2)
        graph = run_robot(graph, start, sequence, part=2, prnt=prnt)
        score = gps_score(graph)
        print('Part 2 result:', score)
    
run(test1, True)
run(test2, True, part=2)
print()
run(test0, part=3)

Initial state
########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: <
########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: ^
########
#.@O.O.#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: ^
########
#.@O.O.#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: >
########
#..@OO.#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: >
########
#...@OO#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: >
########
#...@OO#
##..O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

Move: v
########
#....OO#
##..@..#
#...O..#
#.#.O..#
#...O..#
#...O..#
########

Move: v
########
#....OO#
##..@..#
#...O..#
#.#.O..#
#...O..#
#...O..#
########

Move: <
########
#....OO#
##.@...#
#...O..#
#.#.O..#
#...O..#
#...O..#
########

Move: v
########
#....OO#
##.....#
#..@O..#
#.#.O..#
#...O..#
#...O..#
########

Move: >
########
#....OO#
##.....#
#...@O.#
#.#.O..#
#...O..#
#...O..#
########

Move: >
########
#....

In [4]:
with open('input_day15.txt', 'r') as f:
    data = f.readlines()
    f.close()
    
run(data, part=3)

Part 1 result: 1478649
Part 2 result: 1495455
